In [ ]:
import pandas as pd 
import numpy as np 
import pickle
import xgboost as xgb
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from tqdm import tqdm
import bma_python
from sklearn.metrics import accuracy_score, classification_report, f1_score

In [ ]:
with open('../data/features_training_irony_twitter_semeval.p', 'rb') as handle:
    train = pickle.load(handle)

In [ ]:
X_train_embed = train['bert_embed']

X_train_pp = np.concatenate([train['bert_embed'], train['emoji']['emoji'],
                             np.expand_dims(train['emoji']['emoji_positive'], axis = 1), np.expand_dims(train['emoji']['emoji_negative'], axis = 1),
                         train['punc'], train['onom'], train['init']], axis = 1)

X_train_pos = np.concatenate([train['pos'], train['bert_embed']], axis = 1)

X_train_pp_pos = np.concatenate([train['emoji']['emoji'],np.expand_dims(train['emoji']['emoji_positive'], axis = 1), 
                                 np.expand_dims(train['emoji']['emoji_negative'], axis = 1), train['pos'],train['punc'],
                                 train['onom'], train['init'], train['bert_embed']], axis = 1)

X_train_pp_pos_pol = np.concatenate([train['emoji']['emoji'],np.expand_dims(train['emoji']['emoji_positive'], axis = 1), 
                                 np.expand_dims(train['emoji']['emoji_negative'], axis = 1), train['pos'],train['punc'],
                                 train['onom'], train['init'], train['bert_embed'], train['polarity']], axis = 1)

y = train['label']

In [ ]:
xgb_model = xgb.XGBClassifier(objective = 'binary:logistic', colsample_bytree = 0.8020639316513152, learning_rate = 0.063062124248497, max_depth= 9, min_child_weight= 4, n_estimators= 173, subsample= 0.838253733300651) #bs, pos, pp, polarity
randomf_model = RandomForestClassifier(max_depth = 15, min_samples_leaf = 9, min_samples_split = 4,n_estimators = 13) #bs, pos, pp, pp
hist_model =  HistGradientBoostingClassifier(learning_rate = 0.11711623246492987,max_depth = 14, min_samples_leaf = 13) #bs, pos, polarity, pp 
logi_model = LogisticRegression(C = 0.888809044392379, penalty = 'l2', max_iter=10000)  #bs, pos, polarity, pp 
ada_model = AdaBoostClassifier(learning_rate = 0.683683366733467, n_estimators = 134) #bs, pos pp pola

In [ ]:
model_diz = {'XGB': xgb_model, 'Random F': randomf_model, 'Hist': hist_model, 'Logistic': logi_model, 'Ada': ada_model}

In [ ]:
outputs = bma_python.BMA(model_diz,X_train_pos, y)

In [ ]:
xgb_model = xgb.XGBClassifier(objective = 'binary:logistic', colsample_bytree = 0.8020639316513152, learning_rate = 0.063062124248497, max_depth= 9, min_child_weight= 4, n_estimators= 109, subsample= 0.838253733300651) #bs, pos, pp, polarity
randomf_model = RandomForestClassifier(max_depth = 15, min_samples_leaf = 9, min_samples_split = 14,n_estimators = 13) #bs, pos, pp, pp
hist_model =  HistGradientBoostingClassifier(learning_rate = 0.11711623246492987,max_depth = 14, min_samples_leaf = 13) #bs, pos, polarity, pp 
logi_model = LogisticRegression(C = 0.888809044392379, penalty = 'l2', max_iter=10000)  #bs, pos, polarity, pp 
ada_model = AdaBoostClassifier(learning_rate = 0.683683366733467, n_estimators = 134) #bs, pos pp pola

In [ ]:
xgb_model.fit(X_train_pos, y)
hist_model.fit(X_train_pos, y)
logi_model.fit(X_train_pos, y)
ada_model.fit(X_train_pos, y)
random_f.fit(X_train_pos, y)

## Test Sarcasm

In [ ]:
with open('../data/semeval_test3a_irony_onlysemeval.p', 'rb') as handle:
    semeval = pickle.load(handle)

In [ ]:
semeval_pos = np.concatenate([semeval['pos'], semeval['bert_embed']], axis = 1)
semeval_pp_pos_pol = np.concatenate([semeval['emoji']['emoji'],np.expand_dims(semeval['emoji']['emoji_positive'], axis = 1), 
                                 np.expand_dims(semeval['emoji']['emoji_negative'], axis = 1), semeval['pos'],semeval['punc'],
                         semeval['onom'], semeval['init'], semeval['bert_embed'], semeval['polarity']], axis = 1)
semeval_pp_pos = np.concatenate([semeval['emoji']['emoji'],np.expand_dims(semeval['emoji']['emoji_positive'], axis = 1), 
                                 np.expand_dims(semeval['emoji']['emoji_negative'], axis = 1), semeval['pos'],semeval['punc'],
                                 semeval['onom'], semeval['init'], semeval['bert_embed']], axis = 1)
y_semeval = semeval['label']

In [ ]:
out1 = bma_python.inference_bma(xgb_model, semeval_pos, outputs['Weights'].loc['XGB'])
out2 = bma_python.inference_bma(hist_model, semeval_pos, outputs['Weights'].loc['Hist'])
out3 = bma_python.inference_bma(logi_model, semeval_pos, outputs['Weights'].loc['Logistic'])
out4 = bma_python.inference_bma(ada_model, semeval_pos, outputs['Weights'].loc['Ada'])
out5 = bma_python.inference_bma(random_f, semeval_pos, outputs['Weights'].loc['Random F'])

In [ ]:
sum_test = np.argmax(np.sum((out1,out2, out3, out4), axis = 0), axis = -1)

In [ ]:
print(classification_report(y_semeval, sum_test))

In [ ]:
f1_score(y_semeval, sum_test)